# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [1]:
import boto3
import json

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Clean up campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will list all of the campaigns in your account.

In [3]:
paginator = personalize.get_paginator('list_campaigns')
for paginate_result in paginator.paginate():
    for campaign in paginate_result["campaigns"]:
        print(campaign["campaignArn"])

Look through the list of ARNs to determine which campaigns you want to delete. Then use the code below to delete the campaign by inserting the ARN.

In [ ]:
personalize.delete_campaign(
    campaignArn = "INSERT ARN HERE"
)

## Clean up event trackers

Next, clean up any event trackers. The code below will list all of the event trackers in your account.

In [ ]:
paginator = personalize.get_paginator('list_event_trackers')
for paginate_result in paginator.paginate():
    for event_tracker in paginate_result["eventTrackers"]:
        print(event_tracker["eventTrackerArn"])

Look through the list of ARNs to determine which event tracker you want to delete. Then use the code below to delete the it by inserting the ARN.

In [ ]:
personalize.delete_event_tracker(
    eventTrackerArn = "INSERT ARN HERE"
)

## Clean up solutions

Next, clean up the solutions. The code below will list all of the solutions in your account.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

Look through the list of ARNs to determine which solution you want to delete. Then use the code below to delete the it by inserting the ARN.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## Clean up datasets

Next, clean up the datasets. The code below will list all of the datasets in your account.

In [4]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

arn:aws:personalize:us-east-1:051545784337:dataset/personalize-poc-movielens20m/INTERACTIONS


Look through the list of ARNs to determine which dataset you want to delete. Then use the code below to delete the it by inserting the ARN.

In [5]:
personalize.delete_dataset(
    datasetArn = "arn:aws:personalize:us-east-1:051545784337:dataset/personalize-poc-movielens20m/INTERACTIONS"
)

{'ResponseMetadata': {'RequestId': '5b270bfa-eed8-4c48-a7bf-2bc48ab5404e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 06 Aug 2020 17:42:39 GMT',
   'x-amzn-requestid': '5b270bfa-eed8-4c48-a7bf-2bc48ab5404e',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the schemas

Next, clean up the schemas. The code below will list all of the schemas in your account.

In [6]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

arn:aws:personalize:us-east-1:051545784337:schema/personalize-poc-lastfm-interactions


Look through the list of ARNs to determine which schema you want to delete. Then use the code below to delete the it by inserting the ARN.

In [7]:
personalize.delete_schema(
    schemaArn = "arn:aws:personalize:us-east-1:051545784337:schema/personalize-poc-lastfm-interactions"
)

ResourceInUseException: An error occurred (ResourceInUseException) when calling the DeleteSchema operation: The provided schema is being referenced by one or more datasets

## Clean up the dataset groups

Finally, clean up the dataset group. The code below will list all of the dataset groups in your account.

In [8]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

arn:aws:personalize:us-east-1:051545784337:dataset-group/personalize-poc-movielens20m


Look through the list of ARNs to determine which dataset group you want to delete. Then use the code below to delete the it by inserting the ARN.

In [9]:
personalize.delete_dataset_group(
    datasetGroupArn = "arn:aws:personalize:us-east-1:051545784337:dataset-group/personalize-poc-movielens20m"
)

{'ResponseMetadata': {'RequestId': '50f918a0-e0c5-4a24-b330-1add4c2ae71d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 06 Aug 2020 17:43:22 GMT',
   'x-amzn-requestid': '50f918a0-e0c5-4a24-b330-1add4c2ae71d',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the S3 bucket and IAM role

Optionally, you can delete the IAM role and the S3 bucket which we created throughout the workshop.

Start by listing all of the IAM roles in your account with the code below.

In [10]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

Admin
AmazonSageMaker-ExecutionRole-20200222T182289
AmazonSageMaker-ExecutionRole-20200522T124978
AmazonSageMaker-ExecutionRole-20200616T093520
autoanalyzee-AnalysisAutomationFunctionRole-150JC0W4XWN84
autoanalyzee-CronJobFunctionRole-1KRNWOO9AFRWL
autoanalyzee-CustomResourceExecutionRole-S5ZJH355XS9D
AWSDeepRacerCloudFormationAccessRole
AWSDeepRacerLambdaAccessRole
AWSDeepRacerRoboMakerAccessRole
AWSDeepRacerSageMakerAccessRole
AWSDeepRacerServiceRole
AwsSecurityAudit
AwsSecurityNacundaAudit
AWSServiceRoleForAmazonGuardDuty
AWSServiceRoleForAmazonSageMakerNotebooks
AWSServiceRoleForIsengardControllerRoleInternal
AWSServiceRoleForSupport
AWSServiceRoleForTrustedAdvisor
CustomLabelsServerlessDemo-CustomResourceRole-1FV1PQ9A27TRH
CustomLabelsServerlessDemo-RekognitionInvokeRole-1Y0BXXVXNUD5D
EC2CapacityReservationService
GatedGardenAudit
GatedGardenInternalAudit
InternalAuditInternal
IsengardAuditorRole-DO-NOT-DELETE
IsengardCloudTrailRole
IsengardConfigRole
IsengardRole-DO-NOT-DELETE
la

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [15]:
iam.list_attached_role_policies(
    RoleName = "PersonalizeRolePOC"
)

{'AttachedPolicies': [{'PolicyName': 'AmazonPersonalizeFullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '3dc7d276-6600-4d93-9e5d-22f0f14ae648',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3dc7d276-6600-4d93-9e5d-22f0f14ae648',
   'content-type': 'text/xml',
   'content-length': '571',
   'date': 'Thu, 06 Aug 2020 17:45:10 GMT'},
  'RetryAttempts': 0}}

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [16]:
iam.detach_role_policy(
    RoleName = "PersonalizeRolePOC",
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

{'ResponseMetadata': {'RequestId': '39604df1-a48b-45cb-a1a0-a8ff5d5efb27',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '39604df1-a48b-45cb-a1a0-a8ff5d5efb27',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Thu, 06 Aug 2020 17:45:21 GMT'},
  'RetryAttempts': 0}}

Finally, you should be able to delete the IAM role.

In [17]:
iam.delete_role(
    RoleName = "PersonalizeRolePOC"
)

{'ResponseMetadata': {'RequestId': '1575a325-b081-4527-81fb-b910882924c0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1575a325-b081-4527-81fb-b910882924c0',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Thu, 06 Aug 2020 17:45:24 GMT'},
  'RetryAttempts': 0}}

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.